# Approach
https://sandipanweb.wordpress.com/2017/05/06/some-nlp-spelling-correction-with-noisy-channel-model/

# Build word spelling corrector (autocorrect)

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU, Dot, TimeDistributed, Activation, Embedding
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import json
from nltk.tokenize import word_tokenize
import re
import os
import tarfile
import csv
import pandas as pd
%matplotlib inline

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [203]:
def load_data_with_gt(file_name, delimiter='\t', gt_index=1, prediction_index=0):
    input_texts = []
    gt_texts = []
    for row in open(file_name, encoding='utf8'):
        sents = row.split(delimiter)
        if (len(sents) < 2):
            continue
        input_texts.append(sents[prediction_index])
        gt_texts.append(sents[gt_index])
    return input_texts, gt_texts

In [3]:
def load_raw_data(file_name):
    with open(file_name, 'r') as f:
        return(f.read())

In [4]:
def load_medical_terms(json_file):
    texts = []
    with open(json_file) as f:
        med_terms_dict = json.load(f)
    texts += list(med_terms_dict.keys())
    texts += list(med_terms_dict.values())
    return texts

In [5]:
def load_accidents_terms(file_name):

    f = open(file_name, encoding='utf8')
    line = 0  
    texts = []
    try:
        for r in f:
            for term in r.split('|'):
                    texts += term.replace('\"', '')
    except:
        print('finished')

                
    return texts

In [6]:
def process_word(word):
    # Try to correct the word from known dict
    #word = spell(word)
    # Option 1: Replace special chars and digits
    #processed_word = re.sub(r'[\\\/\-\—\:\[\]\,\.\"\;\%\~\(\)\{\}\$\#\?\●\@\+\-\*\d]', r'', w.lower())
    
    # Option 2: skip all words with special chars or digits
    if(len(re.findall(r'[\\\/\-\—\:\[\]\,\.\"\;\%\~\(\)\{\}\$\#\?\●\@\+\-\*\d]', word.lower())) == 0):
        processed_word = word
    else:
        processed_word = 'UNK'

    # Skip stop words
    #stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]        
    stop_words = []        
    if processed_word in stop_words:
        processed_word = 'UNK'
        
    return processed_word

## Load data

In [198]:
data_path = '../../dat/'
texts = []
noisy_texts = []

In [204]:
# Load tesseract correction

files_list = ['all_ocr_data_2.txt', 
              'field_class_21.txt', 
              'field_class_22.txt', 
              'field_class_23.txt', 
              'field_class_24.txt', 
              'field_class_25.txt', 
              'field_class_26.txt', 
              'field_class_27.txt', 
              'field_class_28.txt', 
              'field_class_29.txt', 
              'field_class_30.txt', 
              'field_class_31.txt', 
              'field_class_32.txt', 
              'field_class_33.txt', 
              'field_class_34.txt', 
              'NL-14622714.txt', 
              'NL-14627449.txt', 
              'NL-14628986.txt', 
              'NL-14631911.txt', 
              'NL-14640007.txt']

#files_list = ['all_ocr_data_2.txt']
#files_list = ['field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt']

for file_name in files_list:
    tess_correction_data = os.path.join(data_path, file_name)
    noisy, gt = load_data_with_gt(tess_correction_data)
    texts += gt
    noisy_texts += noisy
print(len(texts))    
print(len(noisy_texts))    
# Load HW terms

hw_correction_data = os.path.join(data_path, 'handwritten_output.txt')
_, gt = load_data_with_gt(hw_correction_data, delimiter='|', gt_index=0, prediction_index=1)
texts += gt

# Load clean claims forms

num_samples = 10000
file_name = os.path.join(data_path, 'claims.txt')
#texts += load_raw_data(file_name)

# Load Medical Terms dictionary
json_file = os.path.join(data_path, 'abbrevs.json')
texts += load_medical_terms(json_file)

# Load Medical Instruction dictionary
file_name = os.path.join(data_path, 'medical_instructions.txt')
texts += load_raw_data(file_name)

# Load accident terms
file_name = os.path.join(data_path, 'AccidentsL.txt')
texts += load_accidents_terms(file_name)

# Load procedures and tests
file_name = os.path.join(data_path, 'procedures_tests.txt')
texts += load_raw_data(file_name)

# Load Diagnosis Descriptions
file_name = os.path.join(data_path, 'ICD10.csv')
t = pd.read_csv(file_name)
texts += list(t['DESCRIPTION'].values)

2339211
12845
finished


In [9]:
# Sample data
print(len(texts))
for i in range(10):
    print(texts[i], '\n')

1163183
Claim Type: VB Accident - Accidental Injury
 

Who The Reported Event Happened To: Employee/Policyholder
 

Policyholder/Owner Information
 

First Name:
 

Middle Name/Initial:
 

Last Name:
 

Social Security Number:
 

Birth Date:
 

Gender:
 

Language Preference:
 



In [10]:
with open('med.txt', 'w') as f:
    for text in texts:
        f.write(text)
f.close()

In [11]:
!tar -xvf autocorrect/words.tar

words/
words/en_US_GB_CA_mixed.txt
words/big_orig.txt
words/._big.txt
words/big.txt
words/en_US_GB_CA_lower.txt


In [12]:
#f_big_orig = open('words/big_orig.txt', 'r')
f_big_orig = open('med.txt', 'r')
f_med = open('med.txt', 'r')
f_big = open('words/big.txt', 'w')
for line in f_big_orig:
    f_big.write(line + '\n')
for line in f_med:
    f_big.write(line + '\n')
    

f_big_orig.close()
f_big.close()
f_med.close()

In [13]:
!tar -cvf autocorrect/words.tar words

words/
words/en_US_GB_CA_mixed.txt
words/big_orig.txt
words/._big.txt
words/big.txt
words/en_US_GB_CA_lower.txt


In [14]:
!rm -rf words/

# Build n-gram LM

In [15]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from nltk import word_tokenize
from collections import Counter, defaultdict
import random

In [16]:
def lower_case(w):
    if w is not None: w = w.lower() 
    return w

In [17]:
def build_trigram_lm(corpus):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for sentence in corpus:
        sentence = word_tokenize(sentence)
        #print(sentence)
        #print(word_tokenize(sentence))
        #print(list(trigrams(sentence, pad_right=True, pad_left=True)))
        #print(list(trigrams(word_tokenize(sentence), pad_right=True, pad_left=True)))
        for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
            model[(w1, w2)][w3] += 1

    '''
    print(model["what", "the"]["economists"])  # "economists" follows "what the" 2 times
    print(model["what", "the"]["nonexistingword"])  # 0 times
    print(model[None, None]["The"])  # 8839 sentences start with "The"
    '''
    # Let's transform the counts to probabilities
    for w1_w2 in model:
        total_count = float(sum(model[w1_w2].values()))
        for w3 in model[w1_w2]:
            model[w1_w2][w3] /= total_count
    return model


In [40]:
def build_bigram_lm(corpus, vocab_size):
    model = defaultdict(lambda: defaultdict(lambda: 1)) # Default is 1 for laplace smoothing
    vocab = defaultdict(lambda: 0)
    for sentence in corpus:
        sentence = word_tokenize(sentence)
        #print(sentence)
        #print(word_tokenize(sentence))
        #print(list(trigrams(sentence, pad_right=True, pad_left=True)))
        #print(list(trigrams(word_tokenize(sentence), pad_right=True, pad_left=True)))
        for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
            w1 = lower_case(w1)
            w2 = lower_case(w2)
            model[w1][w2] += 1

    # Let's transform the counts to probabilities
    norm_model = defaultdict(lambda: defaultdict(lambda: 1/vocab_size))
    for w1 in model:
        w1 = lower_case(w1)
        total_count = float(sum(model[w1].values())) + vocab_size # Laplace smoothing
        for w2 in model[w1]:
            w2 = lower_case(w2)
            norm_model[w1][w2] = model[w1][w2] / total_count
    return norm_model


In [38]:
def build_unigram_lm(corpus):
    model = defaultdict(lambda: 1) # Default is 1 for laplace smoothing

    for sentence in corpus:
        sentence = word_tokenize(sentence)
        for w1 in sentence:
            w1 = lower_case(w1)
            model[w1] += 1

    # Let's transform the counts to probabilities
    vocab_size = len(model.keys())
    norm_model = defaultdict(lambda: 1/vocab_size) # Default is 1 for laplace smoothing
    for w1 in model:
        w1 = lower_case(w1)
        norm_model[w1] = model[w1] / vocab_size
    return norm_model, vocab_size

In [41]:
p_uni, vocab_size = build_unigram_lm(texts)

In [42]:

p_bi = build_bigram_lm(texts, vocab_size)

In [48]:

from autocorrect import spell
from autocorrect.nlp_parser import NLP_COUNTS
from autocorrect.word import Word, common, exact, known, get_case

In [49]:
import numpy as np
from nltk.tokenize import word_tokenize

In [68]:
def normalize(candidates_dict):
    for candidate in candidates_dict.keys():
        candidates_dict[candidate] /= (sum(candidates_dict.values()) + len(candidates_dict.values()))
    return candidates_dict    

In [188]:
if(0):
    def get_candidates(word):
        w = Word(word)

        candidates = (common([word]) | exact([word]) | known([word]) |
                  known(w.typos()) | common(w.double_typos()) or
                  [word])



        '''
        candidates = (common([word]) | exact([word]) | known([word]) |
                  known(w.typos()) or common(w.double_typos()) or
                  [word])
        '''
        '''
        candidates = (common([word]) or exact([word]) or known([word]) or
                      known(w.typos()) or common(w.double_typos()) or
                      [word])
        '''
        candidates_dict = {}
        for candidate in candidates:
            candidates_dict[candidate] = NLP_COUNTS.get(candidate) + 1
        candidates_dict = normalize(candidates_dict)
        return candidates_dict

In [189]:
def get_candidates(word):
    w = Word(word)
    
    candidates = (common([word]) | exact([word]) | known([word]) |
              known(w.typos()) | common(w.double_typos()) or
              [word])

    w_common = 10
    w_exact = 10
    w_known = 5
    w_known_typo = 2
    w_known_double_typos = 0.005
    w_word = 0.5
    
    tot = w_common + w_exact + w_known + w_known_typo + w_known_double_typos + w_word
    w_common /= tot
    w_exact  /= tot
    w_known  /= tot
    w_known_typo  /= tot
    w_known_double_typos  /= tot
    w_word  /= tot
    
    candidates_dict = {}
    for i, candidate in enumerate(common([word])):
        candidates_dict[candidate] = w_common * (NLP_COUNTS.get(candidate)+1)
    for i, candidate in enumerate(exact([word])):
        candidates_dict[candidate] = w_exact * (NLP_COUNTS.get(candidate)+1)                                  
    for i, candidate in enumerate(known([word])):
        candidates_dict[candidate] = w_known * (NLP_COUNTS.get(candidate)+1)                                  
    for i, candidate in enumerate(known(w.typos())):
        candidates_dict[candidate] = w_known_typo * (NLP_COUNTS.get(candidate)+1)                               
    for i, candidate in enumerate(common(w.double_typos())):
        candidates_dict[candidate] = w_known_double_typos * (NLP_COUNTS.get(candidate)+1) 
    candidates_dict[word] = w_word * (NLP_COUNTS.get(word) + 1)
    
    candidates_dict = normalize(candidates_dict)
    return candidates_dict

In [195]:
special_chars = ['\\', '\/', '\-', '\—' , '\:', '\[', '\]', '\,', '\.', '\"', '\;', '\%', '\~', '\(', '\)', '\{', '\}', '\$', '\&', '\#', '\☒', '\■', '\☐', '\□', '\☑', '\@']
special_chars_s = '[' + ''.join(special_chars) + ']'
def spell_noisy_channel(sentence, p_uni, p_bi):
    sentence = sentence.split(' ')#word_tokenize(sentence)

    wt_1 = []
    corrected = []
    corr_w = spell(sentence[0])
    
    wt_1.append(lower_case(corr_w))
    corrected.append(corr_w)
    for i, w in enumerate(sentence[1:]):
        #print('wt_bar:' + w)
        if((len(re.findall(r'\d+', w))==0) and (len(re.findall(special_chars_s, w))==0)):
            candidates = get_candidates(w)
            prev_score = 0

            for wt_hat in candidates:
                p_wt_hat = p_uni[wt_hat]
                #print('wt_hat:' + wt_hat)
                #print('p_wt_hat:' + str(p_wt_hat))
                '''
                if i > 1:
                    p_wt_hat_given_wt_1 = p_bi[wt_hat][wt_1[i-1]]
                else:
                    p_wt_hat_given_wt_1 = p_wt_hat
                '''
                #print('wt_1:' + wt_1[i-1])
                p_wt_hat_given_wt_1 = p_bi[wt_1[-1]][wt_hat]
                #print('p_wt_hat_given_wt_1:' + str(p_wt_hat_given_wt_1))
                p_wt_hat_given_wt = candidates[wt_hat]
                #print('p_wt_hat_given_wt:' + str(p_wt_hat_given_wt))

                score = p_wt_hat_given_wt * p_wt_hat_given_wt_1 #/ p_wt_hat
                #print('p:' + str(score))
                if score > prev_score: 
                    prev_score = score
                    wt = wt_hat
            
            wt_1.append(wt)
            corrected.append(get_case(w, wt))
        else:
            corrected.append(w)
        
        
                
    return ' '.join(corrected)

In [134]:
'''
def spell_noisy_channel(sentence, p_uni, p_bi):
    sentence = word_tokenize(sentence)

    wt_1 = []
    wt_1.append(spell(sentence[0]))
    for i, w in enumerate(sentence[1:]):
        print('wt_bar:' + w)
        candidates = get_candidates(w)
        prev_score = 0

        for wt_hat in candidates:
            p_wt_hat = p_uni[wt_hat]
            print('wt_hat:' + wt_hat)
            print('p_wt_hat:' + str(p_wt_hat))
            
            #if i > 1:
            #    p_wt_hat_given_wt_1 = p_bi[wt_hat][wt_1[i-1]]
            #else:
            #    p_wt_hat_given_wt_1 = p_wt_hat
            
            print('wt_1:' + wt_1[i-1])
            p_wt_hat_given_wt_1 = p_bi[wt_1[i-1]][wt_hat]
            print('p_wt_hat_given_wt_1:' + str(p_wt_hat_given_wt_1))
            p_wt_hat_given_wt = candidates[wt_hat]
            print('p_wt_hat_given_wt:' + str(p_wt_hat_given_wt))
            
            score = p_wt_hat_given_wt * p_wt_hat_given_wt_1 #/ p_wt_hat
            print('p:' + str(score))
            if score > prev_score: 
                prev_score = score
                wt = wt_hat

        
        wt_1.append(get_case(w, wt))
                
    return ' '.join(wt_1)
'''    

IndentationError: unexpected indent (<ipython-input-134-5424633734e6>, line 17)

In [186]:
#sent = 'First Nane'#'Who The Reported Event Happened To: Employee/Policyholder'#
sent = 'Who The Reported Event Happened To: Employee/Policyholder'
#sent = 'The Reported Event'

In [187]:
c = spell_noisy_channel(sent, p_uni, p_bi)
c

wt_bar:The
wt_hat:The
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.12702657812939597
p:7.777772356686013e-06
wt_hat:the
p_wt_hat:0.2656747489590987
wt_1:who
p_wt_hat_given_wt_1:0.0010338117246412066
p_wt_hat_given_wt:0.02717033929892223
p:2.808901532970554e-05
wt_hat:thee
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.00039880071164748925
p:2.4418363436657436e-08
wt_hat:th
p_wt_hat:0.0002449179524859172
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.00020247166415372566
p:1.2397236355236692e-08
wt_hat:they
p_wt_hat:0.00440852314474651
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.0003421761954655134
p:2.0951273295708633e-08
wt_hat:thm
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.00039917513366484735
p:2.4441289105121685e-08
wt_hat:toe
p_wt_hat:0.08780308596620132
wt_1:who
p_wt_ha

wt_hat:Reported
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.5380196101845587
p:3.294266533091836e-05
wt_hat:reported
p_wt_hat:0.0034900808229243203
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.015331075637174221
p:9.387139136158597e-07
wt_hat:deported
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.0055890133984887215
p:3.4221242949355383e-07
wt_hat:repotted
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.005617999046493035
p:3.439872058837274e-07
wt_hat:reporter
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.005647274198754701
p:3.457797084713875e-07
wt_hat:reposted
p_wt_hat:6.12294881214793e-05
wt_1:who
p_wt_hat_given_wt_1:6.12294881214793e-05
p_wt_hat_given_wt:0.005676843077703374
p:3.475901957937408e-07
wt_hat:resorted
p_wt_hat:6.12294881214793e-05

'Who The Reported Event Happened To: Employee/Policyholder'

In [116]:
special_chars = ['\\', '\/', '\-', '\—' , '\:', '\[', '\]', '\,', '\.', '\"', '\;', '\%', '\~', '\(', '\)', '\{', '\}', '\$', '\&', '\#', '\☒', '\■', '\☐', '\□', '\☑', '\@']
special_chars_s = '[' + ''.join(special_chars) + ']'
def word_spell_correct(decoded_sentence):
    if(decoded_sentence == ''):
        return ''
    corrected_decoded_sentence = ''

    for w in decoded_sentence.split(' '):
        #print(w)
        #if((len(re.findall(r'\d+', w))==0) and not (w in special_chars)):
        if((len(re.findall(r'\d+', w))==0) and (len(re.findall(special_chars_s, w))==0)):
            corrected_decoded_sentence += spell(w) + ' '
        else:
            corrected_decoded_sentence += w + ' '
    return corrected_decoded_sentence.strip()

In [149]:
word_spell_correct(sent)

'Who The Reported Event Happened To: Employee/Policyholder'

In [207]:
for noisy, text in zip(noisy_texts, texts):
    print('GT: ' + text)
    print('Noisy: ' + noisy)
    print('Correction: ' + spell_noisy_channel(noisy, p_uni, p_bi))
    print('-----------')

GT: Claim Type: VB Accident - Accidental Injury

Noisy: Claim Type: VB Accident - Accidental Injury
Correction: Claim Type: OF Accident - Accident Injury
-----------
GT: Who The Reported Event Happened To: Employee/Policyholder

Noisy: “Tho Th 9 Reported Even [ Happen ed To: EnployeefPolicyholder
Correction: Tho Of 9 Reported Open [ Tappen of To: EnployeefPolicyholder
-----------
GT: Policyholder/Owner Information

Noisy: Pol inyhold elm-Chm er [11 form arlon
Correction: Pol inhold elm-Chm of [11 or carbon
-----------
GT: First Name:

Noisy: First Name:
Correction: First Name:
-----------
GT: Middle Name/Initial:

Noisy: Middle Nameﬂnitial:
Correction: Middle Nameﬂnitial:
-----------
GT: Last Name:

Noisy: Last Name:
Correction: Last Name:
-----------
GT: Social Security Number:

Noisy: Social S ecurity Number:
Correction: Social Or security Number:
-----------
GT: Birth Date:

Noisy: Birth Date:
Correction: Birth Date:
-----------
GT: Gender:

Noisy: Gender:
Correction: Gender
-------

Correction: StatefProvince:
-----------
GT: Postal Code:

Noisy: Postal Code:
Correction: Postal Code:
-----------
GT: Country:

Noisy: Comiry
Correction: Comicry
-----------
GT: Date of Visit/Admission:

Noisy: Date ofVisit/‘Admissiorr
Correction: Date ofVisit/‘Admissiorr
-----------
GT: Date of Discharge:

Noisy: Date ofDischarge:
Correction: Date ofDischarge:
-----------
GT: Procedure: Left wrist arthiscopic surgery

Noisy: Procedure: Left wrist arthiscopic surgery
Correction: Procedure Left wrist arthiscopic surgery
-----------
GT: Employment Information

Noisy: Empl Dun ent In formation
Correction: Emp In left In information
-----------
GT: Employer Name:

Noisy: Employer Name:
Correction: Employer Name:
-----------
GT: Policy Number:

Noisy: Policy Number:
Correction: Policy Number:
-----------
GT: Electronic Submission

Noisy: Electronic Submission
Correction: Electronic Submission
-----------
GT: Claim Event Identifier:

Noisy: ClairnEvent Identifier:
Correction: ClairnEvent Id

Correction: So that Unum may evaluate and administer my claims, including providing assistance with return to work. Of such evaluation and administration of claims, the authorization is valid for the years, or the duration of my claim for benefits, whichever is shorter. A understand that once Of Information is disclosed to Unum, any privacy protections established by HIPAA may not apply to the information, but other privacy laws continue to apply. unum may then disclose Of Information only as permitted by law, including, state fraud reporting laws or ii authorized by me.
-----------
GT: I also authorize Unum to disclose My Information to the following persons (for the purpose of reporting claim status or experience, or so that the moment may carry out health care operations, claims payment, administrative or audit functions related to any benefit, plan or claim): any employee benefit plan sponsored by my employer; any person providing services or insurance benefits to (or on behalf of)

KeyboardInterrupt: 

In [208]:
for noisy, text in zip(noisy_texts, texts):
    print('GT: ' + text)
    print('Noisy: ' + noisy)
    print('Correction: ' + word_spell_correct(noisy))
    print('-----------')

GT: Claim Type: VB Accident - Accidental Injury

Noisy: Claim Type: VB Accident - Accidental Injury
Correction: Claim Type: VB Accident - Accidental Injury
-----------
GT: Who The Reported Event Happened To: Employee/Policyholder

Noisy: “Tho Th 9 Reported Even [ Happen ed To: EnployeefPolicyholder
Correction: Tho Th 9 Reported Even [ Happen ed To: EnployeefPolicyholder
-----------
GT: Policyholder/Owner Information

Noisy: Pol inyhold elm-Chm er [11 form arlon
Correction: Pol inhold elm-Chm er [11 form Arlon
-----------
GT: First Name:

Noisy: First Name:
Correction: First Name:
-----------
GT: Middle Name/Initial:

Noisy: Middle Nameﬂnitial:
Correction: Middle Nameﬂnitial:
-----------
GT: Last Name:

Noisy: Last Name:
Correction: Last Name:
-----------
GT: Social Security Number:

Noisy: Social S ecurity Number:
Correction: Social S security Number:
-----------
GT: Birth Date:

Noisy: Birth Date:
Correction: Birth Date:
-----------
GT: Gender:

Noisy: Gender:
Correction: Gender:
----

Correction: ClairnEvent Identifier:
-----------
GT: Submission Date:

Noisy: Submission Date:
Correction: Submission Date:
-----------
GT: Electronically Signed Indicator: Yes

Noisy: Electronically Signed Indicator: Yes
Correction: Electronically Signed Indicator: Yes
-----------
GT: Fraud Statements Reviewed and Electronically

Noisy: Fraud Statements Reviewed and Electronically
Correction: Fraud Statements Reviewed and Electronically
-----------
GT: Signed Date:

Noisy: Signed Date:
Correction: Signed Date:
-----------
GT: unum

Noisy: unumw
Correction: unum
-----------
GT: The Benefits Center

Noisy: C O O The Benefits Center
Correction: C O O The Benefits Center
-----------
GT: Call toll-free Monday through Friday, 8 a.m. to 8 p.m. Eastern Time.

Noisy: Call toll-free Monday through Friday, 8 am. to 8 p.m. Eastern Time.
Correction: Call toll-free Monday through Friday, 8 am. to 8 p.m. Eastern Time.
-----------
GT: Please sign and return this authorization to The Benefits Center at

Correction: UﬂUlTT
-----------
GT: Confirmation of Coverage

Noisy: Conﬁrmation of Coverage
Correction: Confirmation of Coverage
-----------
GT: Employer:

Noisy: Employer:
Correction: Employer:
-----------
GT: Group Policy #:

Noisy: Group Policy #:
Correction: Group Policy #:
-----------
GT: Customer  Policy #:

Noisy: Customer Policy #:
Correction: Customer Policy #:
-----------
GT: EE Name:

Noisy: EE Na me:
Correction: EE Na me:
-----------
GT: The information below is provided to give you a general summary of your coverage and premium as of date of this letter, consistent with the benefits outlined in your certificate.

Noisy: The information below is provided to give you a general summary of your coverage and premium as of the date of this letter, consistent with the beneﬁts outlined in your certificate.
Correction: The information below is provided to give you a general summary of your coverage and premium as of the date of this letter, consistent with the benets outlined in yo

Correction: There in complete tent of the Innate triquetral ligament. Thickening of the proximal eeephelonate ligament is also identiﬁed. suspicions for posttmtnntttie edema. The distal senphoinnnte ligament appears intact. The ulnar aspect of the trinngoinr iibtoenttilnue complex attaches to the dominant audited ﬁnctnro fragment diet the men There is e partial-thickness ulnar side tear efixe triangular ﬁbroenttilngo complex. The vein: and doesn radiocarpal ligaments appear intnet. There is intrasubstance aim-tel involving the extensor carpi ulnaris at the level of the ulnar styloid fractions, consistent with nssoeiet ed tendinopathy (tendinitis vervens tendinosis). No tendon tear or retraction is present. Here is no tmtosynovitis. The [letter and other extensor tendons upper preserved. us do the contents of the mrpai nmnei.
-----------
GT: IMPRESSION:

Noisy: ihil'ltliSSiON: 
Correction: ihil'ltliSSiON: a
-----------
GT: 1. Subacute-appearing ulnar styloid fractures with mild edema id

Correction: Can I request copies information relevant to my claim Yes, you may request copies (free of charge) by contacting us at 1—800-305-0849. Monday through Friday between 8 am - 5 pm (Pacific Time)
-----------
GT: What happens next? If you appeal, we will review our decision and provide you with a written determination. If we continue to deny the payment, coverage, or service requested or you do not receive a timely decision you may be able to request an external review of your claim by an independent third party, who will review the denial and issue a final decision.

Noisy: What happens next? If you appeal. we will review our decision and provide you with a written determination. If we continue to deny the payment. coverage. or service requested or you do not receive a timely decrsion you may be able to request an external review of your claim by an independent third party. who Wlli retiew the denial and issue a final deCiSion.
Correction: What happens next If you appeal. we wi

...

Correction: A strain happens when a muscle is stretched, torn or injured. The pain that you feel is caused by inflammation (swelling) or bruising in the muscle. A strain is not the same as a sprain. A sprain is an injury to a ligament that holds bones together.
-----------
GT: A lumbosacral (low back) strain occurs when twisting, bending or lifting tears the muscle. This leads to stiffness and pain. It is common to experience pain over the muscles around the lower spine but not over the bones. The doctor will usually able to tell where you are tender. A low-back strain is different from a more serious condition called a herniated disk (slipped disk).

Noisy: A lumbosacral {low back) strain occurs when twisting, bending or lifting tears the muscle. This leads to stiffness and pain. It is common to experience pain over the muscles around the lower spine but not over the bones. The doctor will usually able to teli where you are tender. A low-back strain is different from a more serious co

Correction: 0 Culture results may take 2-3 days. We review many culture results and will attempt to contact you if the result: are significant or may change your treatment. But you may also call us back to review culture results. and we encourage you to do so if you have any questions or concerns.
-----------
GT: Prescriptions Written:

Noisy: Prescriptions Written:
Correction: Prescriptions Written:
-----------
GT: Tizanidine 4mg, 1 po every 8 hours as needed for spasm. Dispense Fifteen (15).

Noisy: Tizanidine 4mg, 1 pa every 8 hours as needed for spasm. Dispense Fifteen {15).
Correction: Tizanidine 4mg, 1 pa every 8 hours as needed for spasm. Dispense Fifteen {15).
-----------
GT: Prescriber: Dev, Jasminder Singh

Noisy: Prescriber: Dev, Jasminder Singh
Correction: Prescriber: Dev, Jasminder Singh
-----------
GT: Paper Prescription given to patient

Noisy: Paper Prescription given to patient
Correction: Paper Prescription given to patient
-----------
GT: If side effects develop, suc

Correction: Who Th 2 Reported Event Happened To: EimloyeerPolicyholders Child
-----------
GT: Policyholder/Owner Information

Noisy: Policg'h old er: Owner Information
Correction: Policg'h old er: Owner Information
-----------
GT: First Name:

Noisy: First Name:
Correction: First Name:
-----------
GT: Middle Name/Initial:

Noisy: hﬁddle Name/initial:
Correction: heddle Name/initial:
-----------
GT: Last Name:

Noisy: Last Name:
Correction: Last Name:
-----------
GT: Social Security Number:

Noisy: Social Secun'ty Number:
Correction: Social Secun'ty Number:
-----------
GT: Birth Date:

Noisy: Birth Date:
Correction: Birth Date:
-----------
GT: Gender:

Noisy: Gender:
Correction: Gender:
-----------
GT: Language Preference:

Noisy: Language Preference:
Correction: Language Preference:
-----------
GT: Address Line 1:

Noisy: Address Line 1:
Correction: Address Line 1:
-----------
GT: City:

Noisy: City.
Correction: City.
-----------
GT: State/Province:

Noisy: State/PrOVirice:
Correction:

Correction: Confirmation of Coverage
-----------
GT: The information below is provided to give you a general summary of your coverage and premium consistent with the benefits outlined in your certificate.

Noisy: The information below is provided to give you a general summary of your coverage and premium consistent with the benefits outlined in your certiﬁcate.
Correction: The information below is provided to give you a general summary of your coverage and premium consistent with the benefits outlined in your certiﬁcate.
-----------
GT: Insured Coverage Type Coverage Effective Date

Noisy: Insured Coverme TIE Elfective Coverage Date
Correction: Insured Coverage TIE Effective Coverage Date
-----------
GT: Employee Off-Job Acc June 18, 2012

Noisy: Employee Off—Job Acc June 18, 2012
Correction: Employee Off—Job Acc June 18, 2012
-----------
GT: Spouse Off-Job Acc June 18, 2012

Noisy: Spouse Off—Job Acc June 18, 2012
Correction: Spouse Off—Job Acc June 18, 2012
-----------
GT: Child Off-

KeyboardInterrupt: 